In [7]:
import shutil
import os
import glob

from bs4 import BeautifulSoup as bs
import bs4

import pandas as pd

In [8]:
outdir = '../spreadsheets'
try:
    shutil.rmtree(outdir)
except FileNotFoundError:
    pass
os.mkdir('../spreadsheets')

In [11]:
def parse_node(node):
    if node.name == 'note':
        return None, None, None
    try:
        tag = node['ana']
    except KeyError:
        return None, None, None
    
    type_ = 'object'
    if '#descriptor' in tag.lower():
        type_ = 'descriptor'
        tag = tag.replace('#Descriptor', '').replace('#descriptor', '')
        tag = tag.strip().replace('#', '')
    
    # unpoack into token-tag-type-tuples:
    # 1. mark start of an ad:
    tokens, tags = [], []
    if '#begin' in tag:
        tokens = node.text.split()
        tags = ['O'] * len(tokens)
        tags[0] = 'B'
        types = ['NA'] * len(tokens)
        types[0] = 'begin'        
        return tokens, types, tags
    else:
        tokens = node.text.split()
        tags = [tag.replace('#', '')] * len(tokens)
        types = [type_] * len(tokens)
        return tokens, types, tags

for fn in sorted(glob.glob('../data/annotations/*.xml')):
    with open(fn) as f:
        annotation = bs(f.read(), "lxml").find('text')
        
    tokens, tags, types = [], [], []
    for div in annotation.find_all('div'):
        for ad in div:
            for node in ad:
                if isinstance(node, bs4.element.NavigableString):
                    tokens_ = str(node).split()
                    types_ = tags_ = ['NA'] * len(tokens_)
                    tags_ = ['O'] * len(tokens_)
                elif isinstance(node, bs4.element.Tag):
                    tokens_, types_, tags_ = parse_node(node)

                if not tokens_:
                    continue
                #print(tokens_, tags_)
                #assert len(tokens_) == len(tags_)

                tokens.extend(tokens_)
                types.extend(types_)
                tags.extend(tags_)    
    
    df = pd.DataFrame(zip(tokens, types, tags),
                      columns=['tokens', 'types', 'tags'])
    f_out = os.path.basename(fn).replace('.xml', '')
    df.to_excel(f'{outdir}/{f_out}.xlsx')